In [17]:
import numpy as np
import cv2 

In [50]:
cap = cv2.VideoCapture(0)

# take first frame of the video so we can detect the face at the start. WE only need to detect the face ONE time with mean shift as it will track the clusters for us.
ret,frame = cap.read()

face_cascade = cv2.CascadeClassifier("C:/OpenCV/DATA/haarcascades/haarcascade_frontalface_default.xml")
# face_rectangle is returned as a list of all the numpy arrays where it detects a face
face_rectangle = face_cascade.detectMultiScale(frame)

# turns the 4 points of the rectangle into a tuple so we can tuple unpack it. index[0] means it is the first face detected
face_x,face_y,w,h = tuple(face_rectangle[0])
track_window = (face_x,face_y,w,h) 

# grab face as the roi
roi = frame[face_y:face_y+h, face_x:face_x+w]
# turn to hsv
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
# find histogram to back project it to each frame to calculate mean shift
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0,180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    ret ,frame = cap.read()
    if ret == True:
        
        # Grab the Frame in HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Calculate the Back Projection based off the roi_hist created earlier
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
 #############################################################################################
        # # Apply meanshift to get the new coordinates of the rectangle
        # ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        
        # # Draw the new rectangle on the image
        # x,y,w,h = track_window
        # img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255),5)
 #############################################################################################

        ret, track_window = cv2.CamShift(dst, track_window, term_crit)

        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True,(0,0,255),5)

        cv2.imshow('img2',img2)
        
        
        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
        
    else:
        break
        
cv2.destroyAllWindows()
cap.release()

